In [ ]:
# Based on this paper - https://arxiv.org/pdf/1706.03762.pdf
# Might want to move layer norm inside the residual block - https://arxiv.org/pdf/2002.04745.pdf
# Layer normalization - https://arxiv.org/pdf/1607.06450.pdf
#!pip install torch torchtext sentencepiece datasets

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import sys
import os
import math
sys.path.append(os.path.abspath("../../data"))

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
SP_VOCAB_SIZE = 5000
TRAIN_SIZE = 5000

In [ ]:
from text_data import Opus100DatasetWrapper

class Wrapper(Opus100DatasetWrapper):
    split_lengths = [TRAIN_SIZE, math.floor(TRAIN_SIZE * .1), 100]
    x_length = 40
    target_length = 40

wrapper = Wrapper(SP_VOCAB_SIZE)
datasets = wrapper.generate_datasets(BATCH_SIZE)
train = datasets["train"]
valid = datasets["validation"]

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, input_units, attention_heads, mask=False):
        super(MultiHeadAttention, self).__init__()
        self.input_units = input_units
        self.attention_heads = attention_heads
        self.head_units = int(input_units/attention_heads)
        self.mask = mask
        if self.head_units * self.attention_heads != self.input_units:
            raise Exception("Invalid input units and heads combo")

        k = math.sqrt(1/self.head_units)
        self.query_weights = nn.Parameter(torch.rand(self.attention_heads, input_units, self.head_units) * 2 * k - k)
        self.key_weights = nn.Parameter(torch.rand(self.attention_heads, input_units, self.head_units) * 2 * k - k)
        self.value_weights = nn.Parameter(torch.rand(self.attention_heads, input_units, self.head_units) * 2 * k - k)

    def forward(self, queries, context):
        # convert to 4d tensor with batch_size, attn_heads, seq_len, embedding_dim
        exp_queries = queries.unsqueeze(1).expand(-1, self.attention_heads, -1,-1)
        exp_context = context.unsqueeze(1).expand(-1, self.attention_heads, -1,-1)
        proj_queries = torch.einsum("base, aeh->bash", exp_queries, self.query_weights)
        proj_keys = torch.einsum("base, aeh->bash", exp_context, self.key_weights)
        proj_values = torch.einsum("base, aeh->bash", exp_context, self.value_weights)

        attention = torch.einsum("bash, bahk->bask", proj_queries, torch.transpose(proj_keys, -2, -1)) / np.sqrt(self.head_units)
        if self.mask:
            # Prevent decoder queries from looking at tokens that come after
            # Do this by setting attention to negative infinity, so it is softmaxed to zero in the next step
            mask = torch.zeros((attention.shape[-2], attention.shape[-1]), device=DEVICE)
            mask_indices = np.triu_indices(attention.shape[-2], k=1, m=attention.shape[-1])
            mask[mask_indices] = -torch.inf
            attention += mask

        # Softmax on last dimension
        # Sequence-wise softmax, so attention between one sequence and other sequences sums to 1
        attention = torch.softmax(attention, dim=-1)
        weighted_values = torch.einsum("bash, bahe->base", attention, proj_values)

        # Swap attention head and sequence axis, then reshape to batch, seq, embedding
        weighted_values = weighted_values.view(queries.shape[0], queries.shape[1], -1)
        return weighted_values

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, input_units, attention_heads, hidden_units=2048):
        super(EncoderBlock, self).__init__()
        self.input_units = input_units
        self.attention_heads = attention_heads
        self.hidden_units = hidden_units

        self.mha = MultiHeadAttention(self.input_units, self.attention_heads)
        self.dropouts = nn.ModuleList([nn.Dropout(.1) for _ in range(2)])
        self.linear1 = nn.Linear(self.input_units, hidden_units)
        self.linear2 = nn.Linear(hidden_units, self.input_units)
        self.relu = nn.ReLU()
        self.lns = nn.ModuleList(nn.LayerNorm(self.input_units) for _ in range(2))

    def forward(self, x):
        weighted_values = self.dropouts[0](self.mha(x, x))
        attn_output = self.lns[0](x + weighted_values)

        reprojected = self.dropouts[1](self.linear2(self.relu(self.linear1(attn_output))))
        block_output = self.lns[1](attn_output + reprojected)

        return block_output

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, input_units, attention_heads, hidden_units=2048):
        super(DecoderBlock, self).__init__()
        self.input_units = input_units
        self.attention_heads = attention_heads
        self.hidden_units = hidden_units

        self.in_attn = MultiHeadAttention(self.input_units, self.attention_heads, mask=True)
        self.context_attn = MultiHeadAttention(self.input_units, self.attention_heads)
        self.dropouts = nn.ModuleList([nn.Dropout(.1) for _ in range(3)])
        self.linear1 = nn.Linear(self.input_units, hidden_units)
        self.linear2 = nn.Linear(hidden_units, self.input_units)
        self.relu = nn.ReLU()
        self.lns = nn.ModuleList(nn.LayerNorm(self.input_units) for _ in range(3))

    def forward(self, queries, context):
        weighted_values = self.dropouts[0](self.in_attn(queries, queries))
        attn_output = self.lns[0](queries + weighted_values)

        decoder_values = self.dropouts[1](self.context_attn(attn_output, context))
        decoder_output = self.lns[1](attn_output + decoder_values)

        reprojected = self.dropouts[2](self.linear2(self.relu(self.linear1(decoder_output))))
        block_output = self.lns[2](decoder_output + reprojected)
        return block_output

In [ ]:
class Transformer(nn.Module):
    def __init__(self, input_units, hidden_units, attention_heads, enc_sequence_len, blocks=2):
        super(Transformer, self).__init__()
        self.input_units = input_units
        self.hidden_units = hidden_units
        self.attention_heads = attention_heads
        self.blocks = blocks

        k = math.sqrt(1/self.hidden_units)
        self.embedding = nn.Parameter(torch.rand(hidden_units, self.input_units) * 2 * k - k)
        self.dropout = nn.Dropout(.1)
        self.encoders = nn.ModuleList(EncoderBlock(hidden_units, attention_heads) for _ in range(self.blocks))
        self.decoders = nn.ModuleList(DecoderBlock(hidden_units, attention_heads) for _ in range(self.blocks))
        self.context = None
        self.pos_encoding = torch.from_numpy(self.encoding(enc_sequence_len, self.hidden_units)).to(DEVICE)

    def forward(self, x, y, use_cache=False):
        # TODO: Tie input and output embeddings
        if use_cache and self.context is not None:
            enc_outputs = self.context
        else:
            # 3D with batch, seq, embeddings
            enc_outputs = self.dropout(self.embedding(x) + self.pos_encoding[:x.shape[1]])

            for i in range(self.blocks):
                block_output = self.encoders[i](enc_outputs)
                enc_outputs = block_output

            self.context = enc_outputs

        dec_outputs = self.dropout(self.embedding(y) + self.pos_encoding[:y.shape[1]])
        for i in range(self.blocks):
            block_output = self.decoders[i](dec_outputs, enc_outputs)
            dec_outputs = block_output

        token_vectors = dec_outputs @ self.output_embedding
        return token_vectors

    def encoding(self, seq_len, embed_len):
        encodings = np.zeros((seq_len, embed_len), dtype=np.float32)
        for i in range(seq_len):
            evens = np.arange(0, embed_len, 2)
            odds = np.arange(1, embed_len, 2)
            all = np.power(np.full(embed_len, 10000), np.arange(0, embed_len) / embed_len)

            sin_embed = np.sin(i/all)
            cos_embed = np.cos(i/all)

            sin_embed[odds] = 0
            cos_embed[evens] = 0
            encodings[i,:] = np.sum((sin_embed, cos_embed), axis=0)
        return encodings

In [ ]:
def generate(sequence, pred, target, wrapper):
    prompts = wrapper.decode_batch(sequence.cpu())
    texts = wrapper.decode_batch(torch.argmax(pred, dim=2).cpu())
    correct_texts = wrapper.decode_batch(target.cpu())

    displays = []
    for p, t, ct in zip(prompts, texts, correct_texts):
        displays.append(f"{p} | {ct} | {t}")
    return displays

In [ ]:
from tqdm.auto import tqdm

model = Transformer(wrapper.vocab_size, 512, 8, wrapper.y_length).to(DEVICE)
loss_fn = nn.CrossEntropyLoss(ignore_index=wrapper.pad_token)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
EPOCHS = 100
DISPLAY_BATCHES = 2
OUT_SEQUENCE_LEN = wrapper.y_length
PRINT_VALID = True

for epoch in range(EPOCHS):
    # Run over the training examples
    train_loss = 0
    match_pct = 0
    for batch, (sequence, target, prev_target) in tqdm(enumerate(train)):
        optimizer.zero_grad(set_to_none=True)
        pred = model(sequence.to(DEVICE), prev_target.to(DEVICE))

        # If you use a batch, need to reshape pred to be batch * sequence, embedding_len to be compatible
        # Similar reshape with target to be batch * sequence vector of class indices
        loss = loss_fn(pred.reshape(-1, pred.shape[-1]), target.view(-1).to(DEVICE))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    with torch.no_grad():
        print(f"Epoch {epoch} train loss: {train_loss / len(train)}")
        sents = generate(sequence, pred, target, wrapper)
        for sent in sents[:DISPLAY_BATCHES]:
            print(sent)

        if PRINT_VALID and epoch % 10 ==0:
            # Compute validation loss.  Unless you have a lot of training data, the validation loss won't decrease.
            valid_loss = 0
            # Deactivate dropout layers
            model.eval()
            for batch, (sequence, target, prev_target) in tqdm(enumerate(valid)):
                # Inference token by tokens
                sequence = sequence.to(DEVICE)
                outputs = prev_target[:,0].unsqueeze(1).to(DEVICE)
                pred = None
                model.context = None
                # TODO: Investigate memory leak with valid generation
                for i in range(OUT_SEQUENCE_LEN):
                    pred = model(sequence, outputs, use_cache=True)
                    last_output = torch.argmax(pred, dim=2)
                    outputs = torch.cat((outputs, last_output[:,-1:]), dim=1)
                loss = loss_fn(pred.reshape(-1, pred.shape[-1]), target.view(-1).to(DEVICE))
                valid_loss += loss.item()
            print(f"Valid loss: {valid_loss / len(valid)}")
            sents = generate(sequence, pred, target, wrapper)
            for sent in sents[:DISPLAY_BATCHES]:
                print(sent)
            # Reactivate dropout
            model.train()

In [ ]:
from torchinfo import summary

print(summary(model))

In [125]:
F.one_hot(sequence, num_classes=wrapper.vocab_size)

RuntimeError: one_hot is only applicable to index tensor.